# Interface gap-filling code to MBHB code

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.signal

## 1. Load time-domain LDC data

We are currently interested only in the impact of gaps on the Spritz MBHB-1 dataset. To achieve that we separate the effect of glitches from the effect of gaps in the Spritz data, and we do so by creating ad-hoc datasets with only gaps.

### 1.1. Download and access data

Download LDC 2b *Spritz* MBHB-1 data from the LDC official website (https://lisa-ldc.lal.in2p3.fr/challenge2b) and save them to the `./data/` folder.

In [2]:
# import data
fname = '../../data/LDC2_spritz_mbhb1_training_v2.h5'
# open file with h5py
fid = h5py.File(fname)

In [3]:
# define function to print all attributes of hdf5 file recursively 
# from the LDC quicklook notebooks
def print_attrs(name, obj):
    shift = name.count('/') * '    '
    print(shift + name)
    for key, val in obj.attrs.items():
        print(shift + '    ' + f"{key}: {val}")
# print all dataset attributes (useful to see what's inside the HDF5 file)
fid.visititems(print_attrs)

clean
    clean/tdi
        coord: t
        dt: 5.0
        t0: 8899200.0
        units: dimensionless
gal
    gal/tdi
        coord: t
        dt: 5.0
        t0: 8899200.0
        units: dimensionless
header
    header/date
instru
    instru/config
        instru/config/LISA_AAFILTER_ATTENUATION
        instru/config/LISA_AAFILTER_TRANSITION_BAND_0
        instru/config/LISA_AAFILTER_TRANSITION_BAND_1
        instru/config/LISA_ACC_NOISE_A_LEVEL
        instru/config/LISA_ACC_NOISE_F_KNEE
        instru/config/LISA_ACC_NOISE_USE_PZM
        instru/config/LISA_ADC_RECURSION_ORDER
        instru/config/LISA_BACKLINK_NOISE_A_LEVEL
        instru/config/LISA_BACKLINK_NOISE_F_KNEE
        instru/config/LISA_DEBUG_CLOCK_OFFSETS
        instru/config/LISA_FREQUENCY_OFFSETS_12
        instru/config/LISA_FREQUENCY_OFFSETS_13
        instru/config/LISA_FREQUENCY_OFFSETS_21
        instru/config/LISA_FREQUENCY_OFFSETS_23
        instru/config/LISA_FREQUENCY_OFFSETS_31
        instru/config/LIS

### 1.2. Import the Spritz `tdi` datasets

The Spritz HDF5 file is structured in various groups:
- group `obs` containing `noise + signal + artifacts` (gaps + glitches)
- group `clean` containing `noise + signal`
- group `sky` containing `signal`
- group `noisefree` containing `signal + artifacts` (gaps + glitches)

Each of these groups hosts a dataset `tdi` that contains TDI combinations $X$, $Y$ and $Z$.

The import function `load_tdi_timeseries` also builds the missing TDI dataset:
- TDI dataset containing `noise`
- TDI dataset containing `noise + signal + gaps`

In [4]:
# Print the tdi dataset object
print(fid['obs/tdi'])

<HDF5 dataset "tdi": shape (535680, 1), type "|V32">


In [5]:
from bayesdawn.connect.fillgaps import load_tdi_timeseries

MLDC modules could not be loaded.


Let's load all datasets as numpy-recarray objects in a single dictionary `tdi`:

In [6]:
tdi = load_tdi_timeseries(fname = fname)

Let's take a look at the `'obs'` dataset:

In [7]:
tdi['obs']['X']

array([-1.20843503e-21,  4.66545351e-22,  2.48226674e-21, ...,
       -9.29828606e-21,  5.70021014e-21, -4.30521175e-21])

In [8]:
# dataset is a numpy.recarray object with the length of the data time-series and fields ['t','X','Y','Z']
print(tdi['obs'])
print(tdi['obs'].dtype)

[( 8899200., -1.20843503e-21,  1.03963281e-21, -3.83668942e-21)
 ( 8899205.,  4.66545351e-22, -1.35315602e-21,  7.09159015e-21)
 ( 8899210.,  2.48226674e-21, -1.04844445e-21,  1.09270023e-21) ...
 (11577585., -9.29828606e-21, -6.92159820e-21,  7.19464761e-21)
 (11577590.,  5.70021014e-21, -9.60595940e-22, -4.94114626e-21)
 (11577595., -4.30521175e-21,  8.29780973e-21,  2.00536395e-20)]
[('t', '<f8'), ('X', '<f8'), ('Y', '<f8'), ('Z', '<f8')]


Set up dataset constant values:

In [9]:
# set up dataset constant values

skip = 0 # number of samples to skip
ndata = len(xyzdata['obs'][:]) # length of dataset
dt = xyzdata['obs']['t'][1]-xyzdata['obs']['t'][0] # time step
tobs = ndata * dt # observation time
fs = 1.0/dt # sampling frequency
fmin = np.round(1.0/(tobs),9)+2e-9 # minumum frequency


print('skip','ndata','dt', 'fs', 'tobs\t', 'fmin', sep='\t',end='\n'+60*'-'+'\n')
print(skip, ndata, dt, fs, tobs, fmin, sep='\t')

NameError: name 'xyzdata' is not defined

## 2. Build orthogonal TDI combinations

We can proceed in two different ways to build orthogonal TDI combinations for all the datasets of our interest, i.e. `n+s`, `s`, `n`, `n+s+g` `n+g` data, according to our needs.

1. We can create separate numpy rec-arrays, one for each separate dataset
2. We can create a dictionary containing orthogonal TDI combinations for all imported datasets them


Each of these can then be wired into the Spritz analysis.

In [ ]:
from bayesdawn.connect.fillgaps import build_orthogonal_tdi

In [ ]:
# option 1 - separate numpy rec-arrays
td_data = build_orthogonal_tdi(tdi['obs'])
td_datans = build_orthogonal_tdi(tdi['clean'])
td_datas  = build_orthogonal_tdi(tdi['sky'])
td_datan  = build_orthogonal_tdi(tdi['noise'])
td_datag  = build_orthogonal_tdi(tdi['clean_gapped'])
td_datang = build_orthogonal_tdi(tdi['noise_gapped'])

print(td_datans.dtype.names)

# # option 2 - single dictionary containing all numpy recarrays
# data = build_orthogonal_tdi(tdi)
# print(data.keys())
# print(data['obs'].dtype.names)

In [ ]:
fig, ax = plt.subplots(1,1, dpi=120)
ax.plot(td_datan['t'], td_datan['A'], label='A')
ax.plot(td_datan['t'], td_datan['E'], label='E')
ax.plot(td_datan['t'], td_datan['T'], label='T')
ax.legend()
ax.grid()
ax.set_xlim([1.15e7,1.157e7])

## 3. Define useful functions 

All these will end up in a separate file afterwards.

- `fft_olap_psd`: function that evaluates PSD equivalent to `scipy.signal.welch` and also evaluates FFT at the same time
- `generate_freq_data`: function that applies `fft_olap_psd` to each group inside data and groups results in a single `numpy.recarray` with the same structure of time-domain data.
- `plot_compare_spectra_timeseries`: modified version of `lisabeta_fill_gaps.plot_compare_spectra` function that plots noise spectra and distribution of real/imag part of PSD/noise_model

In [ ]:
from bayesdawn.connect.fillgaps import fft_olap_psd, generate_freq_data, plot_compare_spectra_timeseries

In [ ]:
# Test psd and fft evaluation function
f, psdA, fft_A, _ = fft_olap_psd(td_datan, chan = 'A', plot = True)

In [ ]:
# generate data in the right format for the MBHB search code
# split needs to be set to True

# noise data
datan, psddatan, scalefacn = generate_freq_data(td_datan, split=True, fmin = fmin, win='hanning')
# noise + signal data
datans, psddatans, scalefacns = generate_freq_data(td_datans, split=True, fmin = fmin, win='hanning')
# signal data
data, psddata, scalefac = generate_freq_data(td_datas, split=True, fmin = fmin, win='hanning')

In [ ]:
for param, val in zip(fid['sky/cat'].attrs.keys(),fid['sky/cat'][()]):
    print(param, ':', val)

In [ ]:
# plot data
fig, ax = plt.subplots(1,3,figsize=(15,4),dpi=120, sharey=True)
ax[0].loglog(datan[:,0],np.sqrt(scalefacn*np.square(np.absolute(datan[:,1]+1j*datan[:,2]))), label='A fft norm')
ax[1].loglog(datan[:,0],np.sqrt(scalefacn*np.square(np.absolute(datan[:,3]+1j*datan[:,4]))), label='E fft norm')
ax[2].loglog(datan[:,0],np.sqrt(scalefacn*np.square(np.absolute(datan[:,5]+1j*datan[:,6]))), label='T fft norm')

ax[0].loglog(psddatan[:,0], np.sqrt(psddatan[:,1]), ls='--', label='A PSD')
ax[1].loglog(psddatan[:,0], np.sqrt(psddatan[:,2]), ls='--', label='E PSD')
ax[2].loglog(psddatan[:,0], np.sqrt(psddatan[:,3]), ls='--', label='T PSD')

for i in range(3):
    ax[i].set_xlabel('Frequency (Hz)')
    ax[i].set_ylabel(r'sqrt PSD $S_{TDI}$')
    ax[i].legend()
    ax[i].grid()                                                

For comparing spectra, and compare data with noise models.

Here we compare the three data realizations (noise, signal and noise + signal) and evaluate the gaussianity of the noise realization.

Here we compare the full noise + signal dataset with the only-noise dataset, applying `LISAnoise` as noise model for the TDI noise spectra.

Note that `n + s` data deviate significantly from the Gaussian behavior for A and E, but not as much for T.

In [ ]:
from bayesdawn.connect.fillgaps import plot_compare_spectra_timeseries
from bayesdawn.connect.lisabeta_fill_gaps import plot_compare_spectra

In [ ]:
# plot comparison
plot_compare_spectra_timeseries(data = [td_datans, td_datan] ,
                                freq_bands=[5e-4, 5e-3],
                                fmin = fmin, fmax = 1.5e-2,
                               labels = ['signal+noise', ' noise'], 
                                save = False)

## 4. Data imputation with short custom gaps 


### 4.1 Specify some short gaps (1000 samples) and generate gapped data
Based on the `n` data, we are going to add some gaps to this data realization.

We're going to start with 3 small gaps 20-sample 100-seconds long as in the original notebook to test the code.

In [ ]:
# import lisabeta_fill_gaps as fill_gaps
from bayesdawn.connect.fillgaps import construct_gap_mask

In [ ]:
# we already have time-domain data available, therefore we don't need to construct them starting from the noise spectrum
# we'll just use the LDC time-domain data to inject gaps
td_datans

In [ ]:
SAVE = False
n_gaps = 3
gap_length = 1000

In [ ]:
#construct gap mask
mask = np.ones(ndata)
ts = np.arange(ndata) * dt

maskinfo = construct_gap_mask(ndata,
                              n_gaps = n_gaps,
                              gap_length = gap_length,
                              verbose = True)
mask = maskinfo['mask']

In [ ]:
fig,ax = plt.subplots(1,1,dpi=120)
# ax.plot(maskinfo['mask'])
ax.plot(td_datan['t'],maskinfo['mask'])
ax.grid()
ax.set_xlabel('Time [s]')
ax.set_ylabel('Random gap mask')

In [ ]:
td_datans_masked = td_datans.copy()

for names in td_datans.dtype.names[1:]:
    td_datans_masked[names] = mask * td_datans[names]

for i in range(len(maskinfo['starts'])):
    print('gap', i,
          ':\n\tsamples:', maskinfo['starts'][i],
          '< i <', maskinfo['ends'][i],
          '\n\tseconds:', ts[maskinfo['starts'][i]],
          '< t <', ts[maskinfo['ends'][i]])

### 4.2 Function for viewing/comparing gap data

Given a set of gaps, together with various (eg gapped and ungapped) versions of the TD data, produces comparison plots for each channel around each gap.

In [ ]:
# from lisabeta_fill_gaps import view_gaps
from bayesdawn.connect.fillgaps import view_gaps

In [ ]:
# view_gaps(ts,
#           [y, y_masked, yy, yyt],
#           gapstarts[:3], gapends[:3],
#           labels=['data', 'gapped', 'generated by func', 'transformed and reconstructed'])

view_gaps(ts = td_datans['t'],
          ys = [[td_datans[names] for names in td_datans.dtype.names[1:]]],
          yg = [td_datans_masked[names] for names in td_datans_masked.dtype.names[1:]],
          maskinfo = maskinfo, channels = ['A', 'E'], nwing = 5000,
          labels=['original'])

### 4.3 Define a bayesdawn PSD model class built on the ldc PSD model

In [ ]:
from bayesdawn.connect.fillgaps import LDCModelPSD
from bayesdawn import datamodel, psdmodel
import time

### 4.4 Apply imputation to noise data and compare spectra

#### Define some functions for concretely applying the bayesdawn gap impuation on  a set of FD data channels
- `create_imputation`: Make a set of bayesdawn imputation models from the LDC PSD-0 unequal arms noise model with a specified gap mask
- `update_imputation`: Perform the imputation on a set of FD residual data returning a revised version of the residual data, again in FD with re-drawn gap data.

In [ ]:
from bayesdawn.connect.fillgaps import create_imputation, update_imputation

In [ ]:
# data imputation with iteration loop
# create empty arrays for the imputation
imp_cls = []
psd_cls = []
y_res = []
# create reconstructed data array with empty A, E, T fields
td_data_rec = np.rec.fromarrays([td_datans_masked['t']]+3*[np.zeros(len(td_datans_masked['t']))], 
                               names = td_datans_masked.dtype.names)
# set up flags and variables
names=['A', 'E']
mask = maskinfo['mask']
n_iter = 10
fit_type = 'log_spline'
fit_dof = 15
k = 20

fmin = 1 / t_obs * k

print(' `fmin` value is {:.2e}'.format(fmin))

# perform data imputation

## initialize imputation

for tdi in names:
    t1 = time.time()
    psd_cls, imp_cls, y_res = create_imputation(td_datans_masked, tdi, mask, noise_model = 'spritz')
    td_data_rec[tdi] = y_res
    
    for i in range(n_iter):
        psdmod, imp_cls, y_res = update_imputation(data_rec = td_data_rec, imp_cls = imp_cls, channel = tdi,
                                                  fit_type = fit_type, fit_dof=fit_dof, fmin=fmin)
        td_data_rec[tdi] = y_res
    t2 = time.time()
    print("Imputation for combination " + tdi + " took " + str(t2-t1))
    ax.legend(loc='best')
    ax.grid()

In [ ]:
view_gaps(ts = td_datans['t'],
          ys = [[td_datans[names] for names in td_datans.dtype.names[1:]], 
                [td_data_rec[names] for names in td_datans.dtype.names[1:]]],
          yg = [td_datans_masked[names] for names in td_datans.dtype.names[1:]],
          maskinfo = maskinfo, channels = ['A', 'E'], nwing = 1000,
          labels=['original data', 'reconstructed data', 'gapped data'],
          save = SAVE)

In [ ]:
# generate fft and psd from reconstructed data
fdatan_rec, psddata_rec, _ = generate_freq_data(td_data_rec, split = True,fmin = fmin, win='hanning') 

In [ ]:
# plot comparison
plot_compare_spectra_timeseries(data = [td_datans, td_data_rec],#,data_masked], 
                                  freq_bands=[5e-4, 5e-3],
                                fmin = fmin, fmax = 1.5e-2,
                               labels = ['Downavg original signal+noise', 'Reconstructed signal+noise'],#, 'gapped data'], 
                                save = SAVE)

## 5. Data imputation with Spritz data 
Do this again with the Spritz noise data
### 5.1 Get Spritz gaps data.

In [ ]:
# import lisabeta_fill_gaps as fill_gaps
from bayesdawn.connect.fillgaps import get_ldc_gap_mask

In [ ]:
td_datans

In [ ]:
mask = np.ones(td_datag.shape[0])
ndata = td_datag.shape[0]

spritzgaps = get_ldc_gap_mask(td_datag, mode='index')
gapstarts,gapends= np.vsplit(spritzgaps, 2)

for k in range(gapstarts.shape[1]):
    mask[gapstarts.reshape((2))[k]:gapends.reshape((2))[k]]=0

spritzmask = {'mask': mask,
              'starts': gapstarts.reshape((2)),
              'ends': gapends.reshape((2))}

figname = 'figures/nodownavg_spritzgaps'

In [ ]:
fig,ax = plt.subplots(1,1,dpi=120)
# ax.plot(maskinfo['mask'])
ax.plot(td_datag['t'],spritzmask['mask'])
ax.grid()
ax.set_xlabel('Time [s]')
ax.set_ylabel('Spritz gap mask')

In [ ]:
view_gaps(ts = td_datan['t'],
          ys = [[td_datans[names] for names in td_datans.dtype.names[1:]]],
          yg = [td_datag[names] for names in td_datag.dtype.names[1:]],
          maskinfo = spritzmask, channels = ['A', 'E'],
          labels=['original data'], nwing = 5000, histogram=True)

### 5.3 Apply imputation to noise data and compare spectra

In [ ]:
from bayesdawn.connect.fillgaps import create_imputation, update_imputation

In [ ]:
# data imputation with iteration loop
# create empty arrays for the imputation
imp_cls = []
psd_cls = []
y_res = []
# create reconstructed data array with empty A, E, T fields
td_data_rec = np.rec.fromarrays([td_datag['t']]+3*[np.zeros(len(td_datag['t']))], 
                               names = td_datag.dtype.names)
# set up flags and variables
names=['A', 'E']
mask = spritzmask['mask']
n_iter = 10
fit_type = 'log_spline'
fit_dof = 15
k = 20

fmin = 1 / t_obs * k

print(' `fmin` value is {:.2e}'.format(fmin))

# perform data imputation

## initialize imputation

for tdi in names:
    t1 = time.time()
    psd_cls, imp_cls, y_res = create_imputation(td_datag, tdi, mask, noise_model = 'spritz')
    td_data_rec[tdi] = y_res
    
    for i in range(n_iter):
        psdmod, imp_cls, y_res = update_imputation(data_rec = td_data_rec, imp_cls = imp_cls, channel = tdi,
                                                  fit_type = fit_type, fit_dof=fit_dof, fmin=fmin)
        td_data_rec[tdi] = y_res
    t2 = time.time()
    print("Imputation for combination " + tdi + " took " + str(t2-t1))
    ax.legend(loc='best')
    ax.grid()

In [ ]:
view_gaps(ts = td_datag['t'],
          ys = [[td_datans[names] for names in td_datans.dtype.names[1:]], 
                [td_data_rec[names] for names in td_data_rec.dtype.names[1:]]],
          yg = [td_datag[names] for names in td_datag.dtype.names[1:]],
          maskinfo = spritzmask, channels = ['A', 'E'], nwing = 5000,
          labels=['original', 'reconstructed', 'gapped data'],
          save = SAVE)

In [ ]:
# generate fft and psd from reconstructed data
fdatan_rec, psddata_rec, _ = generate_freq_data(td_data_rec, split = True, fmin = fmin, win='hanning')

In [ ]:
# plot comparison
plot_compare_spectra_timeseries(data = [td_datans, td_data_rec],#,data_masked], 
                                  freq_bands=[5e-4, 5e-3],
                                fmin = fmin, fmax = 1.5e-2,
                               labels = ['Downavg original signal+noise', 'Reconstructed signal+noise'],#, 'gapped data'], 
                                save = SAVE)